In [16]:
# Install required packages
!pip install gradio pandas scikit-learn numpy matplotlib

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [17]:
# 1. Data Preparation and Model Training (Keep this exactly the same)
def generate_synthetic_data(num_samples=10000):
    np.random.seed(42)
    data = {
        'amount': np.random.exponential(scale=500, size=num_samples).round(2),
        'time_of_day': np.random.randint(0, 24, size=num_samples),
        'day_of_week': np.random.randint(0, 7, size=num_samples),
        'transaction_type': np.random.choice(['P2P', 'Merchant', 'Utility'], size=num_samples),
        'sender_history': np.random.randint(1, 100, size=num_samples),
        'receiver_history': np.random.randint(1, 100, size=num_samples),
        'device_change': np.random.choice([0, 1], size=num_samples, p=[0.95, 0.05]),
        'location_change': np.random.choice([0, 1], size=num_samples, p=[0.9, 0.1]),
        'is_fraud': 0
    }

    df = pd.DataFrame(data)
    fraud_indices = np.random.choice(num_samples, size=int(num_samples*0.05), replace=False)

    df.loc[fraud_indices, 'amount'] = np.random.exponential(scale=2000, size=len(fraud_indices)).round(2)
    df.loc[fraud_indices, 'time_of_day'] = np.random.choice([2, 3, 4, 23], size=len(fraud_indices))
    df.loc[fraud_indices, 'device_change'] = np.random.choice([0, 1], size=len(fraud_indices), p=[0.7, 0.3])
    df.loc[fraud_indices, 'location_change'] = np.random.choice([0, 1], size=len(fraud_indices), p=[0.5, 0.5])
    df.loc[fraud_indices, 'is_fraud'] = 1

    return df
    # Generate and prepare data
print("Generating synthetic data...")
df = generate_synthetic_data()
df = pd.get_dummies(df, columns=['transaction_type'], drop_first=True)

# Ensure all expected columns exist
expected_columns = ['transaction_type_Merchant', 'transaction_type_Utility']
for col in expected_columns:
    if col not in df.columns:
        df[col] = 0

Generating synthetic data...


In [18]:
# Feature engineering
df['amount_to_sender_history_ratio'] = df['amount'] / df['sender_history']
df['amount_to_receiver_history_ratio'] = df['amount'] / df['receiver_history']
df['unusual_time'] = df['time_of_day'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4, 23] else 0)

features = ['amount', 'time_of_day', 'day_of_week', 'sender_history',
            'receiver_history', 'device_change', 'location_change',
            'transaction_type_Merchant', 'transaction_type_Utility',
            'amount_to_sender_history_ratio', 'amount_to_receiver_history_ratio',
            'unusual_time']

X = df[features]
y = df['is_fraud']

In [19]:
# Split and scale data
print("Preprocessing data...")
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Preprocessing data...


In [20]:
# Train model
print("Training model...")
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

Training model...


RandomForestClassifier(random_state=42)

In [21]:
# Save model
os.makedirs('model', exist_ok=True)
pickle.dump(model, open('model/fraud_model.pkl', 'wb'))
pickle.dump(scaler, open('model/scaler.pkl', 'wb'))

In [22]:
# 2. Enhanced Gradio Interface with Dark Theme
def predict_fraud(amount, time_of_day, day_of_week, transaction_type,
                 sender_history, receiver_history, device_change, location_change):

    # Prepare input data
    transaction_data = {
        'amount': amount,
        'time_of_day': time_of_day,
        'day_of_week': day_of_week,
        'transaction_type': transaction_type,
        'sender_history': sender_history,
        'receiver_history': receiver_history,
        'device_change': device_change,
        'location_change': location_change
    }

    # Convert to DataFrame
    input_df = pd.DataFrame([transaction_data])
    input_df = pd.get_dummies(input_df, columns=['transaction_type'], drop_first=True)

    # Ensure all columns exist
    for col in features:
        if col not in input_df.columns:
            input_df[col] = 0

    # Feature engineering
    input_df['amount_to_sender_history_ratio'] = input_df['amount'] / input_df['sender_history']
    input_df['amount_to_receiver_history_ratio'] = input_df['amount'] / input_df['receiver_history']
    input_df['unusual_time'] = input_df['time_of_day'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4, 23] else 0)

    X_input = input_df[features]
    X_input_scaled = scaler.transform(X_input)

    # Make prediction
    prediction = model.predict(X_input_scaled)
    probability = model.predict_proba(X_input_scaled)[0][1]

    # Determine risk level
    if probability > 0.7:
        risk_level = "High Risk"
        risk_color = "#ff5252"  # Brighter red for dark theme
    elif probability > 0.3:
        risk_level = "Medium Risk"
        risk_color = "#ffab40"  # Brighter orange for dark theme
    else:
        risk_level = "Low Risk"
        risk_color = "#69f0ae"  # Brighter green for dark theme

    # Create risk meter visualization with dark theme
    plt.style.use('dark_background')
    fig, ax = plt.subplots(figsize=(8, 2), facecolor='#121212')
    ax.barh(['Risk Level'], [probability], color=risk_color)
    ax.set_xlim(0, 1)
    ax.set_xticks([0, 0.3, 0.7, 1])
    ax.set_xticklabels(['0%', '30%', '70%', '100%'], color='white')
    ax.set_title('Fraud Probability Meter', pad=20, color='white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#444')
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.set_facecolor('#121212')

    # Save the plot
    plt.tight_layout()
    plot_path = "risk_meter.png"
    plt.savefig(plot_path, bbox_inches='tight', transparent=True, facecolor='#121212')
    plt.close()

    # Feature importance visualization with dark theme
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)
    top_features = np.array(features)[sorted_idx][-5:]
    top_importance = feature_importance[sorted_idx][-5:]

    fig2, ax2 = plt.subplots(figsize=(8, 4), facecolor='#121212')
    colors = plt.cm.viridis(np.linspace(0.3, 1, len(top_features)))
    ax2.barh(top_features, top_importance, color=colors)
    ax2.set_title('Top Factors Influencing This Prediction', pad=20, color='white')
    ax2.set_xlabel('Importance', color='white')
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_color('#444')
    ax2.spines['bottom'].set_color('#444')
    ax2.tick_params(axis='x', colors='white')
    ax2.tick_params(axis='y', colors='white')
    ax2.set_facecolor('#121212')

    feature_plot_path = "feature_importance.png"
    plt.tight_layout()
    plt.savefig(feature_plot_path, bbox_inches='tight', transparent=True, facecolor='#121212')
    plt.close()

    # Format output with dark theme
    if prediction[0] == 1:
        result = f"""
        <div style='background-color: #1e1e1e; padding: 20px; border-radius: 10px; border-left: 6px solid {risk_color}; margin-bottom: 20px;'>
            <h2 style='color: {risk_color}; margin-top: 0;'>🚨 FRAUD ALERT 🚨</h2>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Risk Level:</strong> <span style='color: {risk_color};'>{risk_level}</span></p>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Confidence:</strong> {probability*100:.2f}%</p>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Recommendation:</strong> DO NOT proceed with this transaction!</p>
        </div>
        """
    else:
        result = f"""
        <div style='background-color: #1e1e1e; padding: 20px; border-radius: 10px; border-left: 6px solid {risk_color}; margin-bottom: 20px;'>
            <h2 style='color: {risk_color}; margin-top: 0;'>✅ Transaction Safe ✅</h2>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Risk Level:</strong> <span style='color: {risk_color};'>{risk_level}</span></p>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Confidence:</strong> {(1-probability)*100:.2f}%</p>
            <p style='font-size: 16px; color: #e0e0e0;'><strong>Recommendation:</strong> You can proceed with this transaction.</p>
        </div>
        """

    return result, plot_path, feature_plot_path

# Dark Theme CSS
css = """
:root {
    --primary: #bb86fc;
    --secondary: #03dac6;
    --accent: #3700b3;
    --background: #121212;
    --surface: #1e1e1e;
    --error: #cf6679;
    --text-primary: #e0e0e0;
    --text-secondary: #a0a0a0;
}

.gradio-container {
    max-width: 900px !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: var(--background) !important;
    color: var(--text-primary) !important;
}

h1 {
    color: var(--primary) !important;
    text-align: center;
    margin-bottom: 20px !important;
}

.description {
    text-align: center;
    margin-bottom: 30px !important;
    color: var(--text-primary) !important;
}

.tab {
    background: var(--surface) !important;
    border-radius: 12px !important;
    padding: 20px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3) !important;
    margin-bottom: 20px !important;
    border: 1px solid #333 !important;
}

.input-column {
    background: var(--surface) !important;
    border-radius: 12px !important;
    padding: 25px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3) !important;
    border: 1px solid #333 !important;
}

.output-column {
    background: var(--surface) !important;
    border-radius: 12px !important;
    padding: 25px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3) !important;
    border: 1px solid #333 !important;
}

label {
    font-weight: 500 !important;
    color: var(--primary) !important;
    margin-bottom: 8px !important;
}

input, select, .radio-group {
    margin-bottom: 20px !important;
    background-color: #2d2d2d !important;
    color: var(--text-primary) !important;
    border: 1px solid #444 !important;
}

.number-input input, .slider-input input {
    border: 1px solid #444 !important;
    border-radius: 8px !important;
    padding: 10px !important;
    background-color: #2d2d2d !important;
    color: var(--text-primary) !important;
}

.slider-input .wrap-inner {
    background: #2d2d2d !important;
}

.radio-group .wrap-inner {
    background: #2d2d2d !important;
    border-radius: 8px !important;
    padding: 10px !important;
    border: 1px solid #444 !important;
}

button {
    background: var(--primary) !important;
    color: #121212 !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 12px 24px !important;
    font-weight: 500 !important;
    transition: all 0.3s ease !important;
}

button:hover {
    background: var(--secondary) !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.3) !important;
    color: #121212 !important;
}

.plot-container {
    margin-top: 20px;
    margin-bottom: 20px;
    border-radius: 8px;
    overflow: hidden;
}

.interpretation {
    background: #2d2d2d;
    padding: 15px;
    border-radius: 8px;
    margin-top: 20px;
    font-size: 14px;
    color: var(--text-primary);
    border: 1px solid #444;
}

.examples {
    background: var(--surface) !important;
    border-radius: 12px !important;
    padding: 20px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3) !important;
    margin-top: 20px !important;
    border: 1px solid #333 !important;
}

.examples .example {
    background: #2d2d2d !important;
    border: 1px solid #444 !important;
    color: var(--text-primary) !important;
}

.examples .example:hover {
    background: #3d3d3d !important;
}

footer {
    text-align: center;
    margin-top: 30px;
    color: var(--text-secondary);
    font-size: 12px;
}

/* Dark scrollbar */
::-webkit-scrollbar {
    width: 8px;
}

::-webkit-scrollbar-track {
    background: #2d2d2d;
}

::-webkit-scrollbar-thumb {
    background: #555;
    border-radius: 4px;
}

::-webkit-scrollbar-thumb:hover {
    background: #666;
}
"""

# Create input components with dark theme styling
inputs = [
    gr.Number(label="Transaction Amount (₹)",
              minimum=0.01,
              elem_classes="number-input"),
    gr.Slider(0, 23, step=1,
              label="Time of Day (0-23 hours)",
              elem_classes="slider-input"),
    gr.Slider(0, 6, step=1,
              label="Day of Week (0=Monday, 6=Sunday)",
              elem_classes="slider-input"),
    gr.Radio(["P2P", "Merchant", "Utility"],
             label="Transaction Type",
             elem_classes="radio-group"),
    gr.Number(label="Your Transaction Count (Last 30 Days)",
              minimum=1,
              value=10,
              elem_classes="number-input"),
    gr.Number(label="Receiver's Transaction Count (If Known)",
              minimum=1,
              value=5,
              elem_classes="number-input"),
    gr.Radio([("No", 0), ("Yes", 1)],
             label="Device Changed Recently?",
             elem_classes="radio-group"),
    gr.Radio([("No", 0), ("Yes", 1)],
             label="Location Changed Recently?",
             elem_classes="radio-group")
]

# Create output components
output_components = [
    gr.HTML(label="Fraud Detection Result"),
    gr.Image(label="Risk Assessment", type="filepath"),
    gr.Image(label="Key Risk Factors", type="filepath")
]

# Create examples
examples = [
    [1500, 3, 5, "P2P", 15, 2, 1, 1],  # Likely fraud
    [500, 14, 2, "Merchant", 30, 50, 0, 0],  # Likely safe
    [2500, 23, 6, "Utility", 5, 1, 1, 1],  # High risk
    [100, 12, 3, "P2P", 40, 30, 0, 0]  # Low risk
]

# Create interface
iface = gr.Interface(
    fn=predict_fraud,
    inputs=inputs,
    outputs=output_components,
    title="UPI Fraud Detection System",
    description="""<div style='text-align: center; margin-bottom: 20px;'>
                    <p style='color: var(--text-primary);'>Check if a UPI transaction might be fraudulent based on transaction details.</p>
                    <p style='font-size: 12px; color: var(--text-secondary);'>This system uses machine learning to analyze patterns and detect potential fraud.</p>
                  </div>""",
    css=css,
    examples=examples,
    allow_flagging="never"
)

# Launch the interface
iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa8935ee678e85490c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
